In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    # queries.shape = [batch_size, num_queries, num_hiddens]
    # keys.shape = [batch_size, num_kvpairs, num_hiddens]
    # values.shape = [batch_size, num_kvpairs, num_hiddens]
    def forward(self, queries, keys, values, valid_lens):
        # self.W_q(queries).shape = [batch_size, num_queries, num_hiddens]
        # queries.shape = [batch_size * num_heads, num_queries, num_hiddens / num_heads]
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        # self.W_k(keys).shape = [batch_size, num_kvpairs, num_hiddens]
        # keys.shape = [batch_size * num_heads, num_kvpairs, num_hiddens / num_heads]
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        # self.W_k(values).shape = [batch_size, num_kvpairs, num_hiddens]
        # values.shape = [batch_size * num_heads, num_kvpairs, num_hiddens / num_heads]
        values = transpose_qkv(self.W_v(values), self.num_heads)
        if valid_lens is not None:
            # valid_lens.shape
            valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)
        # output.shape = [batch_size * num_heads, num_queries, num_hiddens / num_heads]
        output = self.attention(queries, keys, values, valid_lens)
        # output_concat.shape = [batch_size, num_queries, num_hiddens]
        output_concat = transpose_output(output, self.num_heads)
        # 输出.shape = [batch_size, num_queries, num_hiddens]
        return self.W_o(output_concat)


def transpose_qkv(X, num_heads):
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])


# X.shape = [batch_size * num_heads, num_kvpairs, num_hiddens / num_heads]
def transpose_output(X, num_heads):
    # X.shape = [batch_size, num_heads, num_kvpairs, num_hiddens / num_heads]
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    # X.shape = [batch_size, num_kvpairs, num_heads, num_hiddens / num_heads]
    X = X.permute(0, 2, 1, 3)
    # 输出.shape = [batch_size, num_kvpairs, num_hiddens]
    return X.reshape(X.shape[0], X.shape[1], -1)

In [3]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [4]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens = 6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])